In [1]:
import pandas as pd
from processing.utils import filter_data_by_properties, select_structures
from processing.interpolation.Interpolation import apply_interpolation
from processing.dataloader.dataloader import get_dataloader
from processing.create_model.create_model import create_model
from training.hyperparameters.default import get_default_cgcnn_hyperparameters, get_default_painn_hyperparameters, get_default_e3nn_hyperparameters
from training.model_training.trainer import trainer

/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
perovskite_datasets = {}

# dataset_names = ['training_set', 'validation_set', 'test_set', 'holdout_set_B_sites', 'holdout_set_series']
dataset_names = ['training_set', 'validation_set']

for dataset_name in dataset_names:
    perovskite_datasets[dataset_name] = pd.read_json('./data/' + dataset_name + '.json')

In [3]:
property = 'dft_e_hull'

In [4]:
train_data = filter_data_by_properties(perovskite_datasets['training_set'][:100], property)
val_data = filter_data_by_properties(perovskite_datasets['validation_set'][:100], property)

train_data_unrelaxed = select_structures(train_data, 'unrelaxed')
val_data_unrelaxed = select_structures(val_data, 'unrelaxed')

train_data_unrelaxed = apply_interpolation(train_data_unrelaxed, property)
val_data_unrelaxed = apply_interpolation(val_data_unrelaxed, property)

In [5]:
gpu_num = 0
model_type = 'e3nn_contrastive'
OUTDIR = 'saved_models/' + model_type + '/Scrap2'

In [6]:
train_loader = get_dataloader(train_data_unrelaxed, prop="dft_e_hull", model_type=model_type, batch_size=10, interpolation=True)
val_loader = get_dataloader(val_data_unrelaxed, prop="dft_e_hull", model_type=model_type, batch_size=10, interpolation=True)

100%|██████████| 100/100 [00:00<00:00, 2379.46it/s]


In [7]:
hyperparameters = get_default_e3nn_hyperparameters()
hyperparameters['num_hidden_layer'] = 1
hyperparameters['num_conv'] = 1
hyperparameters['num_radial_neurons'] = 10
print(hyperparameters)

model, Normalizer = create_model(model_type, train_loader, hyperparameters=hyperparameters, prop=property)

{'MaxEpochs': 100, 'log_lr': -3, 'reduceLR_patience': 10, 'len_embedding_feature_vector': 64, 'num_hidden_feature': 128, 'num_hidden_layer': 1, 'multiplicity_irreps': 32, 'num_conv': 1, 'num_radical_basis': 10, 'num_radial_neurons': 10}


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
